### Librerias

In [ ]:
#%pip install pyodbc

In [ ]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,desc,sum,to_date,date_format,concat_ws,format_string,to_timestamp,year,when,lit,month
from pyspark.sql import Window
import pandas as pd

# Crear una SparkSession
# Crear una SparkSession
spark = SparkSession.builder.appName("AzureSQL").getOrCreate()

## Credenciales BD
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Credenciales.txt"
keys = pd.read_csv(inpath, sep = ',')
display(keys)

# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value

# Configurar las propiedades de la conexión
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database_datamart}"
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

# Conexion al Blob
spark.conf.set(clave_blob, access_key_blob)

In [ ]:
## TABLA SALES
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/Completo_ventas/Ds_G_sales.parquet'
sales = spark.read.parquet(inpath)
#print(sales.count())
#display(sales.limit(100))

sales.createOrReplaceTempView("sales_vw")

In [ ]:
# tmp = sales.select('SUB_TOPIC').distinct()
# display(tmp)

In [ ]:
# tmp = sales.withColumn('TOWN_NW',col("TOWN")).\
#     select('TOWN','SITE','TOWN_NW').distinct()
# # #tmp = sales.filter(col('T_PRODUCT_ID') == '10229196910867') #eladio cali
# # #print(tmp.count())
# display(tmp)

In [ ]:

# tmp = spark.sql(
#     """
# SELECT DISTINCT
#   TOPIC,
#   TOPIC_N,
#   TOPIC_PPTO
# FROM sales_vw
# WHERE YEAR(Fecha_evento) IN (2020)
# """)

# #print(tmp.count()) 
# display(tmp)

In [ ]:
# temp = sales.select('T_PRODUCT_ID','FILE_NUMBER').\
#     filter( col('product_family').isin('Paquete','Competición','Evento','Paquetes',
# 		'Pases con fecha y hora','Pase de visita','Pases sin fecha')).distinct().\
#     orderBy("T_PRODUCT_ID")

# print(temp.count())

# # Agregar una columna que indique si es duplicado o no
# window_spec = Window().partitionBy("FILE_NUMBER").orderBy("FILE_NUMBER")

# df_spark = temp.withColumn("es_duplicado", count("FILE_NUMBER").over(window_spec) > 1)

# # Filtrar solo las filas duplicadas
# duplicados = df_spark.filter("es_duplicado")#.select('Evento').distinct()
# #duplicados = duplicados.withColumn('Evento_N', lit('Hola'))
# # DF = df3.toPandas()

# # # Verificar duplicados en la columna 'llave_identificadora'
# # duplicados = DF[DF.duplicated(subset=['Key_evento'], keep=False)]

# #df_spark = spark.createDataFrame(duplicados)
# display(duplicados)
# print(duplicados.count())

In [ ]:
# temp = sales.filter((col('FILE_NUMBER') == '9394807') #& col('T_OPERATION_TYPE').isin('SALE', 'REFUND')
#                     ).\
#     select(['ORGANIZATION','FILE_NUMBER','T_PRODUCT_ID','PRODUCT','NET_SOLD_TKT_AMT_ITX','ORDER_NUMBER','T_OPERATION_TYPE','t_operation_kind']) 
# #temp = contacts.filter(col('CONTACT_NUMBER') == '9022570') #
# display(temp.limit(100))

In [ ]:
# temp = sales.filter(col('t_operation_kind').isin('ORDER_OVERHEADS','DELIVERY_OVERHEADS')).\
#     select(['FILE_NUMBER','PRODUCT','NET_SOLD_TKT_AMT_ITX','ORDER_NUMBER','T_OPERATION_TYPE','t_operation_kind']).\
#     distinct() 

# # Agregar una columna que indique si es duplicado o no
# window_spec = Window().partitionBy("FILE_NUMBER").orderBy("FILE_NUMBER")

# df_spark = temp.withColumn("es_duplicado", count("FILE_NUMBER").over(window_spec) > 1)

# # Filtrar solo las filas duplicadas
# duplicados = df_spark.filter("es_duplicado")#.select('Evento').distinct()

# display(duplicados.limit(100))
# #display(temp.limit(100))

In [ ]:
eventos = spark.sql("""
    --el numero de eventos se contabiliza aparte debido a que para llegar a el, se deben realizar filtros distintos 
    --with num_eventos_ as (
	select 
	  s.T_ORDER_CONTACT_ID, 
	  s.PRODUCT,
	  s.TEAM_1,
	  s.TEAM_2,
	  s.TOPIC,
      s.SUB_TOPIC,
      '' as PRODUCT_maestra,
	  s.TOPIC_N, 
	  s.SUB_TOPIC_N,
	  s.PRODUCT_DATE_TIME,
      sum(s.NET_SOLD_TKT_AMT_ITX) RECAUDO,
      year(s.PRODUCT_DATE_TIME) as ANIO
	from sales_vw s
	where product_family IN ('Competición','Evento','Pase de visita')
		AND UPPER(s.PRODUCT) NOT LIKE '%PRUEBA%'
		AND s.PRODUCT is not null
		AND T_ORDER_CONTACT_ID is not null
		--AND s.PRODUCT = 'DESDE QUE INVENTARON LAS EXCUSAS'
  		--AND UPPER(PRODUCT_STATE) != 'SUSPENDIDO'
    group by T_ORDER_CONTACT_ID,
    	s.PRODUCT,
     	TEAM_1,TEAM_2,TOPIC,SUB_TOPIC,
        TOPIC_N,SUB_TOPIC_N,
        PRODUCT_DATE_TIME,ANIO
	order by T_ORDER_CONTACT_ID,
   		ANIO,
 		s.PRODUCT
	""")

print(eventos.count())

7375106


In [ ]:
# #temp = eventos.select('PRODUCT','TOPIC','SUB_TOPIC','TOPIC_N','SUB_TOPIC_N','ANIO').distinct().filter(eventos.TOPIC_N == 'SIN DATO')
# temp = eventos.select('PRODUCT','TOPIC','SUB_TOPIC','TOPIC_N','SUB_TOPIC_N','ANIO').distinct().filter(eventos.ANIO == '2024')
# display(temp)

In [ ]:
# display(eventos.limit(100))

In [ ]:
# resultado = eventos.groupBy("TOPIC_N").count()
# display(resultado)

In [ ]:
# Impresion
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Eventos/"
# 2023_fcf
join_df_coalesced = eventos.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)
#join_df_coalesced.write.mode("overwrite").orc(path_parquet_snappy)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_G_Categoria_Eventos.parquet")

True

In [ ]:
# from datetime import datetime

# # Obtener el año actual
# anio_actual = datetime.now().year
# anio_ant = anio_actual - 1

# anio_actual = str(anio_actual)
# anio_ant = str(anio_ant)
# # Mostrar el año actual
# #print(anio_actual,anio_ant)


### Lectura de bases

In [ ]:
# ## Ventas anio anterior
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/sales/' + anio_ant + '/Ds_S3_Sales_' + anio_ant + '.parquet/'
# sales_ant = spark.read.parquet(inpath)
# print(sales_ant.count())

# ## Ventas anio actual
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/sales/' + anio_actual + '/S3/Ds_S3_Sales_' + anio_actual + '.parquet/'
# sales_actual = spark.read.parquet(inpath)


# ### 2023 ###
# #Ds_S1_Sales_2023_11=Ds_B_Sales_2023_11_df.select(columnas)
# sales_actual=sales_actual
# print(sales_actual.count())

# ### Union de Ventas anio anterior-actual
# sales = sales_ant.union(sales_actual)
# print(sales.count())

In [ ]:
## Ventas 2019
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2019/Ds_S_PaymentMethods_2019.parquet/'
payments_2019 = spark.read.parquet(inpath)
#print(payments_2019.count())
#payments_2019.createOrReplaceTempView("payments_2019_vw")

## payments_2020
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2020/Ds_S_PaymentMethods_2020.parquet/'
payments_2020 = spark.read.parquet(inpath)
#print(payments_2020.count())
#payments_2020.createOrReplaceTempView("payments_2020_vw")

## payments_2021
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2021/Ds_S_PaymentMethods_2021.parquet/'
payments_2021 = spark.read.parquet(inpath)
#print(payments_2021.count())
#payments_2021.createOrReplaceTempView("payments_2021_vw")

## payments_2022
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2022/Ds_S_PaymentMethods_2022.parquet/'
payments_2022 = spark.read.parquet(inpath)
#print(payments_2022.count()) 
#payments_2022.createOrReplaceTempView("payments_2022_vw")

## payments_2023
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2023/Ds_S_PaymentMethods_2023.parquet/'
payments_2023 = spark.read.parquet(inpath)
#print(payments_2023.count())
#payments_2023.createOrReplaceTempView("payments_2023_vw")


## payments_2024
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/2024/Ds_S_PaymentMethods_2024.parquet/'
payments_2024 = spark.read.parquet(inpath)
#print(payments_2024.count())
#payments_2023.createOrReplaceTempView("payments_2023_vw")

In [ ]:
payments = payments_2019.union(payments_2020).union(payments_2021).union(payments_2022).union(payments_2023).union(payments_2024)
#sales = sales_2021.union(sales_2022).union(sales_2023)

#print(sales_2023.count())
payments.createOrReplaceTempView("payments_vw")

In [ ]:
dim_payments = spark.sql("""
with temp1 as (
	select 
	FILE_NUMBER,
	PAYMENT_METHOD,
	sum(NET_SOLD_AMT_ITX) recaudo
	from payments_vw
	group by FILE_NUMBER,PAYMENT_METHOD)

select distinct 
FILE_NUMBER,
FIRST_VALUE(PAYMENT_METHOD) OVER (PARTITION BY FILE_NUMBER ORDER BY recaudo desc) AS PAYMENT_METHOD
from temp1
order by FILE_NUMBER
""")

# vista de dim_payments
dim_payments.createOrReplaceTempView("dim_payments_vw")
#dim_payments.count()

In [ ]:
# tmp = payments_2024.filter(month(col('REFERENCE_DATE')) == '3')
# print(tmp.count())
# tmp.createOrReplaceTempView("payments_2024_vw")

In [ ]:
# tmp = dim_payments.\
#     groupBy("PAYMENT_METHOD").count().orderBy(col("count").desc())
# display(tmp)

In [ ]:
# ## Payments anio anterior
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/' + anio_ant + '/Ds_S_PaymentMethods_' + anio_ant + '.parquet/'
# payments_ant = spark.read.parquet(inpath)
# print(payments_ant.count())

# ## Payments anio actual
# inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/payment/' + anio_actual + '/Ds_S_PaymentMethods_' + anio_actual + '.parquet/'
# payments_actual = spark.read.parquet(inpath)
# print(payments_actual.count())


# ### Union de Ventas anio anterior-actual
# payments = payments_ant.union(payments_actual)
# print(payments.count())

In [ ]:
## Contacts
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Contacts/Ds_S_Contacts.parquet"
contacts = spark.read.parquet(inpath)
print(contacts.count())

contacts.createOrReplaceTempView("contacts_vw")

3844843


In [ ]:
# #temp = contacts.filter(col('T_CONTACT_ID') == '10229522833023')
# temp = contacts.filter(col('CONTACT_NUMBER') == '9033085')
# display(temp.limit(20))

In [ ]:
# temp = contacts.select('T_CONTACT_ID').distinct()
# print(temp.count())

### Consulta de audiencias

In [ ]:
# tmp = dim_payments.filter("PAYMENT_METHOD").count().orderBy(col("count").desc())
# display(tmp)

In [ ]:
historico = spark.sql("""

WITH tmp11 as (
    SELECT DISTINCT
    Key_evento,
    Key_aforo,
    Nuevo_aforo as Aforo
    FROM sales_vw ),
 
 tmp12 as (
	SELECT 
    Key_evento,
    SUM(Aforo) as Aforo
	FROM tmp11
  GROUP BY Key_evento)

SELECT
s.FILE_NUMBER as Expediente,
s.T_ORGANIZ_ID as Codigo_organismo,
s.ORGANIZATION as Organismo,
s.REFERENCE_DATE as Fecha_Compra,
s.PRODUCT_DATE_TIME as Fecha_hora_evento,
s.Fecha_evento,
s.Hora_evento,
s.Key_evento,
c.Edad,
c.Rango_de_edad,
c.Genero, 
CASE 
    WHEN c.CIUDAD IS NULL THEN 'SIN DATO'
    ELSE c.CIUDAD
  END AS Ciudad_del_cliente,
c.REGION as Region_del_cliente,
c.DEPARTAMENTO as Departamento_del_cliente,
c.CAPITAL as Ciudad_capital_del_cliente,
s.SALES_CHANNEL_TYPE as Canal_de_venta,
s.product_family as Familia_producto,
s.T_PRODUCT_ID as Codigo_evento,
s.PRODUCT_N as Evento,
--s.PRODUCT_CODE,
s.T_PERFORMANCE_ID as Sesion,
s.TEAM_1,
s.TEAM_2_N as TEAM_2,
s.TOPIC as Tema,
s.SUB_TOPIC as Subtema,
s.TOWN_N as Ciudad_venue,
s.SITE as Venue,
--p.PAYMENT_METHOD as Metodo_de_pago,
s.TOPIC_N as Tema_gral,
s.SUB_TOPIC_N as Subtema_gral,
s.Nuevo_aforo as Aforo_localidad,
t.Aforo,
s.LOGICAL_SEAT_CATEGORY,
s.T_OPERATION_TYPE,
s.T_OPERATION_KIND,
SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados,
SUM(s.NET_SOLD_C_QTY) AS Cortesias,
SUM(s.NET_SOLD_TKT_AMT_ITX) AS Recaudo
FROM sales_vw s
LEFT JOIN contacts_vw c
  ON s.T_ORDER_CONTACT_ID = c.T_CONTACT_ID
LEFT JOIN tmp12 t
  ON s.Key_evento = t.Key_evento
--LEFT JOIN dim_payments_vw p
--  ON s.FILE_NUMBER = p.FILE_NUMBER
--LEFT JOIN Maestra_subtopic_2_vw m
--  ON s.PRODUCT = m.PRODUCT
WHERE --YEAR(s.REFERENCE_DATE) >= 2022
    s.T_OPERATION_TYPE IN ('SALE','REFUND')
		AND s.T_OPERATION_KIND IN ('SINGLE_ENTRY','PRODUCT_COMPOSITION','SIMPLE_PRODUCT')--'COMPOSED_PRODUCT' 
		AND s.PRODUCT_CODE != 'CAN_INS'
    --AND s.product_family IN ('Paquete','Competición','Evento','Paquetes',
        --'Pases con fecha y hora','Pase de visita','Pases sin fecha') 
GROUP BY
s.FILE_NUMBER,s.T_ORGANIZ_ID,s.ORGANIZATION,s.REFERENCE_DATE,s.PRODUCT_DATE_TIME,
s.Fecha_evento,
s.Hora_evento,
s.Key_evento,c.Edad,c.Rango_de_edad,c.Genero,c.CIUDAD,c.REGION,c.DEPARTAMENTO,c.CAPITAL,s.SALES_CHANNEL_TYPE,s.product_family,
s.T_PRODUCT_ID,s.PRODUCT_N,--s.PRODUCT_CODE,
s.T_PERFORMANCE_ID,s.TEAM_1,s.TEAM_2_N,s.TOPIC,s.SUB_TOPIC,s.TOWN_N,s.SITE,--p.PAYMENT_METHOD,
s.TOPIC_N,s.SUB_TOPIC_N,s.Nuevo_aforo,t.Aforo,s.DN_QUOTA,s.LOGICAL_SEAT_CATEGORY,s.T_OPERATION_TYPE,s.T_OPERATION_KIND
"""
)

#historico_.count()
# display(historico_.limit(10))

In [ ]:
# Impresion
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Modelos_analitica/Base_historico/"
# 2023_fcf
join_df_coalesced = historico.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_G_Base_historico.parquet")

True

In [ ]:
# handling = spark.sql(
#     """
# WITH temp as (
#     SELECT
#     s.FILE_NUMBER as Expediente,
#     SUM(s.NET_SOLD_TKT_AMT_ITX) as Handling_fee
#     FROM sales_vw s
#     WHERE s.T_OPERATION_TYPE IN ('SALE','REFUND')
#         AND s.T_OPERATION_KIND IN ('DELIVERY_OVERHEADS','ORDER_OVERHEADS')--'COMPOSED_PRODUCT' 
#     --		AND s.PRODUCT_CODE != 'CAN_INS'
#         --AND s.product_family IN ('Paquete','Competición','Evento','Paquetes',
#             --'Pases con fecha y hora','Pase de visita','Pases sin fecha') 
#     GROUP BY
#     s.FILE_NUMBER,s.REFERENCE_DATE
# ),

# WITH temp as ()

# SELECT
# CONCAT(Expediente,'|',Mes_anio) as Key_handling,
# SUM(Handling_fee) Handling_fee
# FROM temp
# GROUP BY Expediente,Mes_anio
# ORDER BY Expediente,Mes_anio
# """
# )

# print(handling.count())
# display(handling.limit(10))


In [ ]:
audiencias_merch = spark.sql(
    """
SELECT
s.FILE_NUMBER,
s.ORGANIZATION as Organismo,
s.T_ORDER_CONTACT_ID as Id_contacto,
c.FIRSTNAME as NOMBRE,
c.LASTNAME as APELLIDO,
s.ORDER_CONTACT_NUMBER as COD_CONTACTO,
c.FECHA_DE_NACIMIENTO,
s.REFERENCE_DATE as Fecha_Compra,
s.PRODUCT_DATE_TIME as Fecha_hora_evento,
s.Fecha_evento,
--s.Hora_evento,
s.Key_evento,
s.Key_aforo,
s.ORDER_CREATION_DATE AS Hora_de_compra,
c.Edad,
c.Rango_de_edad,
c.Genero,
c.EMAIL as Correo,
c.NAT_NUMBER_CELLPHONE as Movil,
c.MAIN_ADDR_LINE1 as Direccion, 
CASE 
    WHEN c.CIUDAD IS NULL THEN 'SIN DATO'
    ELSE c.CIUDAD
  END AS Ciudad_del_cliente,
c.CAPITAL as Ciudad_capital_del_cliente,
c.DEPARTAMENTO as Departamento_del_cliente,
c.PAIS as Pais_del_cliente,
s.SALES_CHANNEL as Canal_de_venta,
s.SALES_CHANNEL_N,
s.SALES_CHANNEL_TYPE as Subcanal,
s.product_family as Familia_producto,
s.T_PRODUCT_ID as Codigo_evento,
s.PRODUCT_N as Evento,
s.PRODUCT_CODE,
s.LICENCE_NUMBER as Codigo_pulep,
s.PROMOTION as Promocion,
s.LOGICAL_SEAT_CATEGORY as Categoria_Logica,
s.TOWN_N as Ciudad_venue,
s.SITE as Venue,
p.PAYMENT_METHOD as Metodo_de_pago,
s.TOPIC_PPTO as Tema_gral,
s.SUB_TOPIC_N as Subtema_gral,
s.Nuevo_aforo as Aforo,
s.PERFORMANCE_QUOTA as Aforo_old,
s.T_OPERATION_TYPE,
s.T_OPERATION_KIND,
s.INVOICE_CONTACT_FMT_NAME as Promotor,
SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados,
SUM(s.NET_SOLD_C_QTY) AS Cortesias,
SUM(s.NET_SOLD_TKT_AMT_ITX) AS Recaudo
FROM sales_vw s
LEFT JOIN contacts_vw c
  ON s.T_ORDER_CONTACT_ID = c.T_CONTACT_ID
LEFT JOIN dim_payments_vw p
  ON s.FILE_NUMBER = p.FILE_NUMBER
--LEFT JOIN Maestra_subtopic_2_vw m
--  ON s.PRODUCT = m.PRODUCT
WHERE --YEAR(s.REFERENCE_DATE) >= 2022
  s.T_OPERATION_TYPE IN ('SALE','REFUND')
	AND s.T_OPERATION_KIND IN ('SINGLE_ENTRY','PRODUCT_COMPOSITION','SIMPLE_PRODUCT')--'COMPOSED_PRODUCT' 
	AND s.PRODUCT_CODE != 'CAN_INS'
    --AND s.product_family IN ('Paquete','Competición','Evento','Paquetes',
        --'Pases con fecha y hora','Pase de visita','Pases sin fecha') 
GROUP BY
s.FILE_NUMBER,s.ORGANIZATION,s.T_ORDER_CONTACT_ID,c.FIRSTNAME,c.LASTNAME,s.ORDER_CONTACT_NUMBER,c.FECHA_DE_NACIMIENTO,s.REFERENCE_DATE,s.PRODUCT_DATE_TIME,s.Fecha_evento,--s.Hora_evento,
s.Key_evento,s.Key_aforo,
s.ORDER_CREATION_DATE,c.Edad,c.Rango_de_edad,c.Genero,c.EMAIL,c.NAT_NUMBER_CELLPHONE,c.ADDRESS_SALUTATION,c.MAIN_ADDR_LINE1,c.CIUDAD,c.CAPITAL,c.DEPARTAMENTO,c.PAIS,s.SALES_CHANNEL,s.SALES_CHANNEL_N,s.SALES_CHANNEL_TYPE,s.product_family,s.T_PRODUCT_ID,s.PRODUCT_N,s.PRODUCT_CODE,s.LICENCE_NUMBER,s.PROMOTION,s.LOGICAL_SEAT_CATEGORY,s.TOWN_N,s.SITE,p.PAYMENT_METHOD,s.TOPIC_PPTO,s.SUB_TOPIC_N,s.Nuevo_aforo,s.PERFORMANCE_QUOTA,s.T_OPERATION_TYPE,s.T_OPERATION_KIND,s.INVOICE_CONTACT_FMT_NAME
"""
)

audiencias_merch.count()
# display(audiencias_merch.limit(10))
# print(audiencias_merch.head(5))

8889655

In [ ]:
from pyspark.sql.functions import col

archivo_csv = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/temp/metodo/metodos_pago_2.csv"
df = spark.read.csv(archivo_csv, header=True, inferSchema=True, sep=';',encoding="ISO-8859-1")

# Renombra la columna en `df` para evitar conflictos de nombres
df = df.withColumnRenamed("Metodo_de_pago", "Metodo_de_pago_2")

# Realiza un join basado en la columna "Metodo_de_pago" utilizando alias
audiencias_merch = audiencias_merch.alias("a").join(
    df.alias("b"),
    col("a.Metodo_de_pago") == col("b.Metodo_de_pago_2"),
    "left_outer"
)

In [ ]:
from pyspark.sql.functions import col, initcap
# Obtiene todas las columnas
todas_las_columnas = audiencias_merch.columns

# Selecciona solo las columnas de tipo cadena
columnas_cadena = [col_name for col_name, dtype in audiencias_merch.dtypes if dtype == "string"]

# Lista de campos que deseas omitir
campos_a_omitir = ["CORREO",'Tema',"Tema_gral",'T_OPERATION_TYPE','T_OPERATION_KIND','PRODUCT_CODE']

# Aplica las transformaciones a las columnas de tipo cadena
audiencias_merch = audiencias_merch.select(
    *[initcap(col(column)).alias(column) if column in columnas_cadena and column not in campos_a_omitir else col(column) for column in todas_las_columnas]

)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-591896717185790>, line 3
      1 from pyspark.sql.functions import col, initcap
      2 # Obtiene todas las columnas
----> 3 todas_las_columnas = audiencias_merch.columns
      5 # Selecciona solo las columnas de tipo cadena
      6 columnas_cadena = [col_name for col_name, dtype in audiencias_merch.dtypes if dtype == "string"]

NameError: name 'audiencias_merch' is not defined

In [ ]:
# Impresion
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Audiencias_mercadeo/"
# 2023_fcf 
join_df_coalesced = audiencias_merch.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_G_audiencias_mercadeo.parquet")

True

In [ ]:
# audiencias = spark.sql(
#     """
# SELECT
# s.FILE_NUMBER,
# s.T_ORGANIZ_ID as Codigo_organismo,
# s.ORGANIZATION as Organismo,
# s.T_ORDER_CONTACT_ID as Id_contacto,
# c.FIRSTNAME as NOMBRE,
# c.LASTNAME as APELLIDO,
# c.ID_NUMBER as Cedula,
# s.ORDER_CONTACT_NUMBER as COD_CONTACTO,
# c.FECHA_DE_NACIMIENTO,
# s.REFERENCE_DATE as Fecha_Compra,
# s.PRODUCT_DATE_TIME as Fecha_hora_evento,
# s.Fecha_evento,
# --s.Hora_evento,
# s.Key_evento,
# s.ORDER_CREATION_DATE AS Hora_de_compra,
# c.Edad,
# c.Rango_de_edad,
# c.EMAIL as CORREO,
# c.NAT_NUMBER_CELLPHONE as MoVIL,
# c.Genero,
# c.MAIN_ADDR_LINE1 as Direccion, 
# CASE 
#     WHEN c.CIUDAD IS NULL THEN 'SIN DATO'
#     ELSE c.CIUDAD
#   END AS Ciudad_del_cliente,
# --c.CIUDAD as Ciudad_del_cliente,
# c.REGION as Region_del_cliente,
# c.DEPARTAMENTO as Departamento_del_cliente,
# c.CAPITAL as Ciudad_capital_del_cliente,
# c.PAIS as Pais_del_cliente,
# s.SALES_CHANNEL as Canal_de_venta,
# s.SALES_CHANNEL_N,
# s.SALES_CHANNEL_TYPE as Subcanal,
# s.product_family as Familia_producto,
# s.T_PRODUCT_ID as Codigo_evento,
# s.PRODUCT as Evento,
# s.T_PERFORMANCE_ID as Sesion,
# --CONCAT(FORMAT_NUMBER(s.T_PRODUCT_ID, '0'), '_',FORMAT_NUMBER(s.T_PERFORMANCE_ID, '0')) AS Key_evento,
# CONCAT(s.FILE_NUMBER,'|',CONCAT(YEAR(s.REFERENCE_DATE),'_',MONTH(s.REFERENCE_DATE))) AS Key_handling,
# s.TEAM_1,
# s.TEAM_2_N as TEAM_2,
# s.LICENCE_NUMBER as Codigo_pulep,
# s.PROMOTION as Promocion,
# s.LOGICAL_SEAT_CATEGORY as Categoria_Logica,
# --s.T_SITE_ID as Codigo_del_venue,
# s.TOPIC as Tema,
# s.SUB_TOPIC as Subtema,
# s.SEASON as Temporada,
# s.TOWN as Ciudad_venue,
# c.ADDRESS_SALUTATION as Tratamiento,
# s.SITE as Venue,
# s.VENUE_N as Venue_N,
# s.Tipo_venue,
# p.PAYMENT_METHOD as Metodo_de_pago,
# s.PRODUCT_CODE,
# --m.PRODUCT as evento_maestra,
# --CASE WHEN m.NUEVO_TOPIC IS NULL
# --  THEN s.TOPIC_N
# --  ELSE m.NUEVO_TOPIC END AS Tema_gral,
# --CASE WHEN m.NUEVO_SUBTOPIC IS NULL
# --  THEN s.SUB_TOPIC_N
# --  ELSE m.NUEVO_SUBTOPIC END AS Subtema_gral,
# s.TOPIC_PPTO as Tema_gral,
# s.SUB_TOPIC_N as Subtema_gral,
# s.Nuevo_aforo as Aforo,
# s.PERFORMANCE_QUOTA as Aforo_old,
# s.T_OPERATION_TYPE,
# s.T_OPERATION_KIND,
# s.AUDIENCE_CATEGORY,
# s.T_INVOICE_CONTACT_ID as Id_promotor,
# s.INVOICE_CONTACT_FMT_NAME as Promotor,
# SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados,
# SUM(s.NET_SOLD_C_QTY) AS Cortesias,
# SUM(s.NET_SOLD_TKT_AMT_ITX) AS Recaudo,
# SUM(s.CHARGE_AMT_ITX) as Booking_fee,
# SUM(s.CHARGE_AMT_ETX) as Booking_fee_sin_imp,
# SUM(s.INSURED_AMT_ITX) as INSURED_AMT_ITX,
# SUM(s.INSURED_AMT_ETX) as INSURED_AMT_ETX
# FROM sales_vw s
# LEFT JOIN contacts_vw c
#   ON s.T_ORDER_CONTACT_ID = c.T_CONTACT_ID
# LEFT JOIN dim_payments_vw p
#   ON s.FILE_NUMBER = p.FILE_NUMBER
# --LEFT JOIN Maestra_subtopic_2_vw m
# --  ON s.PRODUCT = m.PRODUCT
# WHERE YEAR(s.REFERENCE_DATE) >= 2022
#     AND s.T_OPERATION_TYPE IN ('SALE','REFUND')
# --		AND s.T_OPERATION_KIND IN ('SINGLE_ENTRY','PRODUCT_COMPOSITION','SIMPLE_PRODUCT')--'COMPOSED_PRODUCT' 
# --		AND s.PRODUCT_CODE != 'CAN_INS'
#     --AND s.product_family IN ('Paquete','Competición','Evento','Paquetes',
#         --'Pases con fecha y hora','Pase de visita','Pases sin fecha') 
# GROUP BY
# s.FILE_NUMBER,s.T_ORGANIZ_ID,s.ORGANIZATION,s.T_ORDER_CONTACT_ID,c.FIRSTNAME,c.LASTNAME,c.ID_NUMBER,s.ORDER_CONTACT_NUMBER,c.FECHA_DE_NACIMIENTO,s.REFERENCE_DATE,s.PRODUCT_DATE_TIME,
# s.Fecha_evento,s.Key_evento,--s.Hora_evento,
# s.ORDER_CREATION_DATE,c.Edad,c.Rango_de_edad,c.EMAIL,c.NAT_NUMBER_CELLPHONE,c.Genero,c.ADDRESS_SALUTATION,c.MAIN_ADDR_LINE1,c.CIUDAD,c.REGION,c.DEPARTAMENTO,c.CAPITAL,c.PAIS,s.SALES_CHANNEL,s.SALES_CHANNEL_N,s.SALES_CHANNEL_TYPE,s.product_family,s.T_PRODUCT_ID,s.PRODUCT,s.T_PERFORMANCE_ID,s.TEAM_1,s.TEAM_2_N,s.LICENCE_NUMBER,s.PROMOTION,s.LOGICAL_SEAT_CATEGORY,s.TOPIC,s.SUB_TOPIC,s.SEASON,s.TOWN,c.ADDRESS_SALUTATION,s.SITE,s.VENUE_N,s.Tipo_venue,p.PAYMENT_METHOD,s.PRODUCT_CODE,s.TOPIC_PPTO,s.SUB_TOPIC_N,s.Nuevo_aforo,s.PERFORMANCE_QUOTA,s.T_OPERATION_TYPE,s.T_OPERATION_KIND,s.AUDIENCE_CATEGORY,s.T_INVOICE_CONTACT_ID,s.INVOICE_CONTACT_FMT_NAME
# """
# )

# audiencias.count()
# #display(audiencias.limit(10))
# audiencias.createOrReplaceTempView("audiencias_vw")

In [ ]:
audiencias = spark.sql(
    """

WITH Ranked_Values AS (
  -- Ajuste asistencias Daniel Hernandez 7-may-2024
  SELECT
    s.FILE_NUMBER,
    s.INSURED_AMT_ETX,
    ROW_NUMBER() OVER (PARTITION BY s.FILE_NUMBER ORDER BY s.INSURED_AMT_ETX DESC) AS rn
  FROM sales_vw s
  WHERE s.INSURED_AMT_ETX != 0 AND YEAR(s.REFERENCE_DATE) >= 2022 AND s.T_OPERATION_TYPE IN ('SALE','REFUND')
)
SELECT
s.FILE_NUMBER,
s.T_ORGANIZ_ID as Codigo_organismo,
s.ORGANIZATION as Organismo,
s.T_ORDER_CONTACT_ID as Id_contacto,
c.FIRSTNAME as NOMBRE,
c.LASTNAME as APELLIDO,
c.ID_NUMBER as Cedula,
s.ORDER_CONTACT_NUMBER as COD_CONTACTO,
c.FECHA_DE_NACIMIENTO,
s.REFERENCE_DATE as Fecha_Compra,
s.PRODUCT_DATE_TIME as Fecha_hora_evento,
s.Fecha_evento,
--s.Hora_evento,
s.Key_evento,
s.Key_aforo,
s.ORDER_CREATION_DATE AS Hora_de_compra,
c.Edad,
c.Rango_de_edad,
c.EMAIL as CORREO,
c.NAT_NUMBER_CELLPHONE as MoVIL,
c.Genero,
c.MAIN_ADDR_LINE1 as Direccion, 
CASE 
    WHEN c.CIUDAD IS NULL THEN 'SIN DATO'
    ELSE c.CIUDAD
  END AS Ciudad_del_cliente,
--c.CIUDAD as Ciudad_del_cliente,
c.REGION as Region_del_cliente,
c.DEPARTAMENTO as Departamento_del_cliente,
c.CAPITAL as Ciudad_capital_del_cliente,
c.PAIS as Pais_del_cliente,
s.SALES_CHANNEL as Canal_de_venta,
s.SALES_CHANNEL_N,
s.SALES_CHANNEL_TYPE as Subcanal,
s.product_family as Familia_producto,
s.T_PRODUCT_ID as Codigo_evento,
s.PRODUCT_N as Evento,
s.T_PERFORMANCE_ID as Sesion,
--CONCAT(FORMAT_NUMBER(s.T_PRODUCT_ID, '0'), '_',FORMAT_NUMBER(s.T_PERFORMANCE_ID, '0')) AS Key_evento,
s.Key_handling,
s.TEAM_1,
s.TEAM_2_N as TEAM_2,
s.LICENCE_NUMBER as Codigo_pulep,
s.PROMOTION as Promocion,
s.LOGICAL_SEAT_CATEGORY as Categoria_Logica,
--s.T_SITE_ID as Codigo_del_venue,
s.TOPIC as Tema,
s.SUB_TOPIC as Subtema,
s.SEASON as Temporada,
s.TOWN_N as Ciudad_venue,
c.ADDRESS_SALUTATION as Tratamiento,
s.SITE as Venue,
s.VENUE_N as Venue_N,
s.Tipo_venue,
p.PAYMENT_METHOD as Metodo_de_pago,
s.PRODUCT_CODE,
--m.PRODUCT as evento_maestra,
--CASE WHEN m.NUEVO_TOPIC IS NULL
--  THEN s.TOPIC_N
--  ELSE m.NUEVO_TOPIC END AS Tema_gral,
--CASE WHEN m.NUEVO_SUBTOPIC IS NULL
--  THEN s.SUB_TOPIC_N
--  ELSE m.NUEVO_SUBTOPIC END AS Subtema_gral,
s.TOPIC_PPTO as Tema_gral,
s.SUB_TOPIC_N as Subtema_gral,
s.Nuevo_aforo as Aforo,
s.PERFORMANCE_QUOTA as Aforo_old,
s.T_OPERATION_TYPE,
s.T_OPERATION_KIND,
s.AUDIENCE_CATEGORY,
s.T_INVOICE_CONTACT_ID as Id_promotor,
s.INVOICE_CONTACT_FMT_NAME as Promotor,
SUM(s.NET_SOLD_T_QTY)- SUM(s.NET_SOLD_C_QTY) as Tickets_comprados,
SUM(s.NET_SOLD_C_QTY) AS Cortesias,
SUM(s.NET_SOLD_TKT_AMT_ITX) AS Recaudo,
SUM(s.CHARGE_AMT_ITX) as Booking_fee,
SUM(s.CHARGE_AMT_ETX) as Booking_fee_sin_imp,
SUM(s.INSURED_AMT_ITX) as INSURED_AMT_ITX,
SUM(s.Hfee) as Hfee,
rv.INSURED_AMT_ETX AS INSURED_AMT_ETX,
s.t_file_state
FROM sales_vw s
LEFT JOIN contacts_vw c
  ON s.T_ORDER_CONTACT_ID = c.T_CONTACT_ID
LEFT JOIN dim_payments_vw p
  ON s.FILE_NUMBER = p.FILE_NUMBER
LEFT JOIN Ranked_Values rv
  ON s.FILE_NUMBER = rv.FILE_NUMBER AND rv.rn = 1
--LEFT JOIN Maestra_subtopic_2_vw m
--  ON s.PRODUCT = m.PRODUCT
WHERE YEAR(s.REFERENCE_DATE) >= 2022
    AND s.T_OPERATION_TYPE IN ('SALE','REFUND')
--		AND s.T_OPERATION_KIND IN ('SINGLE_ENTRY','PRODUCT_COMPOSITION','SIMPLE_PRODUCT')--'COMPOSED_PRODUCT' 
--		AND s.PRODUCT_CODE != 'CAN_INS'
    --AND s.product_family IN ('Paquete','Competición','Evento','Paquetes',
        --'Pases con fecha y hora','Pase de visita','Pases sin fecha') 
GROUP BY
s.FILE_NUMBER,s.T_ORGANIZ_ID,s.ORGANIZATION,s.T_ORDER_CONTACT_ID,c.FIRSTNAME,c.LASTNAME,c.ID_NUMBER,s.ORDER_CONTACT_NUMBER,c.FECHA_DE_NACIMIENTO,s.REFERENCE_DATE,s.PRODUCT_DATE_TIME,
s.Fecha_evento,s.Key_evento,s.Key_aforo,--s.Hora_evento,
s.ORDER_CREATION_DATE,c.Edad,c.Rango_de_edad,c.EMAIL,c.NAT_NUMBER_CELLPHONE,c.Genero,c.ADDRESS_SALUTATION,c.MAIN_ADDR_LINE1,c.CIUDAD,c.REGION,c.DEPARTAMENTO,c.CAPITAL,c.PAIS,s.SALES_CHANNEL,s.SALES_CHANNEL_N,s.SALES_CHANNEL_TYPE,s.product_family,s.T_PRODUCT_ID,s.PRODUCT_N,s.T_PERFORMANCE_ID,s.Key_handling,s.TEAM_1,s.TEAM_2_N,s.LICENCE_NUMBER,s.PROMOTION,s.LOGICAL_SEAT_CATEGORY,s.TOPIC,s.SUB_TOPIC,s.SEASON,s.TOWN_N,c.ADDRESS_SALUTATION,s.SITE,s.VENUE_N,s.Tipo_venue,p.PAYMENT_METHOD,s.PRODUCT_CODE,s.TOPIC_PPTO,s.SUB_TOPIC_N,s.Nuevo_aforo,s.PERFORMANCE_QUOTA,s.T_OPERATION_TYPE,s.T_OPERATION_KIND,s.AUDIENCE_CATEGORY,s.T_INVOICE_CONTACT_ID,s.INVOICE_CONTACT_FMT_NAME, rv.INSURED_AMT_ETX,s.t_file_state
"""
)

# audiencias.count()
#display(audiencias.limit(10))
audiencias.createOrReplaceTempView("audiencias_vw")

## AJUSTE DEL CAMPO (INSURED_AMT_ETX)

In [ ]:
audiencias = audiencias.withColumn(
    "INSURED_AMT_ETX",
    when(audiencias.INSURED_AMT_ITX == 0, 0)
    .otherwise(audiencias.INSURED_AMT_ETX)
)

In [ ]:
from pyspark.sql.functions import col, when, max as max_

# Definir una ventana que particione por FILE_NUMBER
window_spec = Window.partitionBy("FILE_NUMBER")

# Agregar una columna que contenga el valor de 'recaudo' donde las condiciones específicas se cumplen
audiencias = audiencias.withColumn(
    "recaudo_CAN_INS_SALE",
    max_(when((col("PRODUCT_CODE") == "CAN_INS") & (col("T_OPERATION_TYPE") == "SALE"), col("recaudo"))).over(window_spec)
)

audiencias = audiencias.withColumn(
    "recaudo_CAN_INS_REFUND",
    max_(when((col("PRODUCT_CODE") == "CAN_INS") & (col("T_OPERATION_TYPE") == "REFUND"), col("recaudo"))).over(window_spec)
)

audiencias = audiencias.withColumn(
    "INSURED_AMT_ETX",
    when(
        (col("PRODUCT_CODE") != "CAN_INS") & 
        (col("PRODUCT_CODE").isNotNull()) & 
        (col("T_OPERATION_TYPE") == "SALE") &
        (col("recaudo_CAN_INS_SALE").isNotNull()),
        col("recaudo_CAN_INS_SALE")/1.19
    ).when(
        (col("PRODUCT_CODE") != "CAN_INS") & 
        (col("PRODUCT_CODE").isNotNull()) & 
        (col("T_OPERATION_TYPE") == "REFUND") &
        (col("recaudo_CAN_INS_REFUND").isNotNull()),
        col("recaudo_CAN_INS_REFUND")/1.19        
    ).otherwise(col("INSURED_AMT_ETX"))
)


audiencias = audiencias.withColumn(
    "INSURED_AMT_ETX",
    when(
        (col("T_OPERATION_KIND") == "PRODUCT_COMPOSITION"),0       
    ).otherwise(col("INSURED_AMT_ETX"))
)

# Eliminar la columna temporal 'recaudo_CAN_INS_SALE'
audiencias = audiencias.drop("recaudo_CAN_INS_SALE","recaudo_CAN_INS_REFUND")


In [ ]:
from pyspark.sql.functions import col, when, max as max_

## REFUNDS DEL SEGURO NO VALIDOS 
# Marcar las filas que cumplen las condiciones
audiencias = audiencias.withColumn(
    "valid_refund",
    when((col("PRODUCT_CODE") == "CAN_INS") & (col("T_OPERATION_TYPE") == "REFUND"), 1).otherwise(0)
)

# Usar una ventana para determinar si existe al menos una fila válida en cada grupo FILE_NUMBER
window_spec = Window.partitionBy("FILE_NUMBER")
audiencias = audiencias.withColumn(
    "has_valid_refund",
    max_("valid_refund").over(window_spec)
)

## SE SIGUE CON EL CODIGO ORIGINAL
# Filtrar por las condiciones específicas
filtered_df = audiencias.filter(
    audiencias.T_OPERATION_KIND.isin("SINGLE_PRODUCT", "SINGLE_ENTRY", "COMPOSED_PRODUCT")
)

# Usar una ventana para determinar el valor máximo a aplicar en insured_amt_etx
window_spec = Window.partitionBy("FILE_NUMBER", "Codigo_evento")


# Preparar un DataFrame con los valores máximos por grupo
max_values_df = (
    filtered_df.withColumn(
        "max_insured_amt_etx",
        max_("insured_amt_etx").over(window_spec)
    ).withColumn(
        "max_insured_amt_etx",
        when(col("has_valid_refund") == 0, 0).otherwise(col("max_insured_amt_etx"))
    ).select(col("max_insured_amt_etx"), col("file_number").alias("FILE_NUMBER_B"), col("Codigo_evento").alias("Codigo_evento_B"), col("max_insured_amt_etx").alias("MAX_INSURED_AMT_ETX_B"),col("T_OPERATION_TYPE").alias("T_OPERATION_TYPE_B")).distinct() 
    # Asegurarse de que no haya duplicados en este DataFrame
)

# Hacer join con el DataFrame original para actualizar los valores
audiencias = audiencias.join(
    max_values_df,
    (audiencias["file_number"] == max_values_df["file_number_B"]) &
    (audiencias["Codigo_evento"] == max_values_df["Codigo_evento_B"]) &
    (audiencias["T_OPERATION_TYPE"] == max_values_df["T_OPERATION_TYPE_B"]) &
    (max_values_df["T_OPERATION_TYPE_B"] == "REFUND"),
    how="left"
)

# Actualizar los valores de insured_amt_etx con los nuevos valores máximos
audiencias = audiencias.withColumn(
    "insured_amt_etx",
    when(
        col("max_insured_amt_etx").isNotNull(),
        col("max_insured_amt_etx")
    ).otherwise(col("insured_amt_etx"))
)

# Si aún observas duplicados, aplicar distinct aquí puede ayudar, pero con precaución
# audiencias = audiencias.distinct()

audiencias = audiencias.withColumn(
    "insured_amt_etx",
    when(
        (col("t_operation_type") == "REFUND") &
        (col("max_insured_amt_etx").isNotNull()),                                                 
        col("insured_amt_etx") * -1
    ).otherwise(col("insured_amt_etx"))
)

col_eliminar = ['FILE_NUMBER_B','Codigo_evento_B','MAX_INSURED_AMT_ETX_B','T_OPERATION_TYPE_B',"valid_refund", "has_valid_refund","max_insured_amt_etx"]

audiencias = audiencias.drop(*col_eliminar)

In [ ]:
# # Modificación inicial para establecer 'recaudo_condicional'
audiencias = audiencias.withColumn(
    'recaudo_condicional',
    when(
        (col('product_code').isNotNull()) & (col('product_code') != 'CAN_INS') & (col('T_OPERATION_KIND') != 'PRODUCT_COMPOSITION'),
        col('recaudo')
    ).otherwise(0)
)

# Definir la ventana para la creacion de columna recaudo_2
window_spec_2 = Window.partitionBy("FILE_NUMBER",'T_OPERATION_TYPE') #Creacion ventana adicional por caso file = 11972945

# Calcular la suma acumulada solo de valores positivos
audiencias = audiencias.withColumn(
    'recaudo_suma_acumulada_positivos_2',
    sum(col('recaudo_condicional')).over(window_spec_2)
)

# Definir una ventana que considere todas las filas anteriores desde el inicio del DataFrame
windowSpec = Window.partitionBy('File_Number')

# Calcular la suma acumulada solo de valores positivos
audiencias = audiencias.withColumn(
    'recaudo_suma_acumulada_positivos',
    sum(when(col('recaudo_condicional') > 0, col('recaudo_condicional')).otherwise(0)).over(windowSpec)
)



# Calcular la suma acumulada solo de valores negativos
audiencias = audiencias.withColumn(
    'recaudo_suma_acumulada_negativos',
    sum(when(col('recaudo_condicional') < 0, col('recaudo_condicional')).otherwise(0)).over(windowSpec)
)

audiencias = audiencias.withColumn(
    'recaudo_suma_neg_pos',
    col('recaudo_suma_acumulada_positivos') + col('recaudo_suma_acumulada_negativos')
)

audiencias = audiencias.withColumn(
    "recaudo_2",
    sum(col("recaudo_suma_acumulada_positivos")).over(window_spec)
)

# Usar los valores acumulados para ajustar 'INSURED_AMT_ETX'
audiencias = audiencias.withColumn(
    'INSURED_AMT_ETX',
    when(
        col('recaudo_suma_neg_pos') == 0,
        when(
            col('recaudo_condicional') > 0,
            (col('INSURED_AMT_ETX') / col('recaudo_suma_acumulada_positivos')) * col('recaudo')
        ).when(
            col('recaudo_condicional') < 0,
            (col('INSURED_AMT_ETX') / col('recaudo_suma_acumulada_negativos')) * col('recaudo')
        ).otherwise(col('INSURED_AMT_ETX'))
    ).otherwise(
            (col('INSURED_AMT_ETX') / col('recaudo_suma_acumulada_positivos_2')) * col('recaudo')
    )
)


col_eliminar = ['recaudo_condicional','recaudo_suma_acumulada','recaudo_suma_acumulada_positivos','recaudo_suma_acumulada_negativos','recaudo_2','recaudo_suma_neg_pos','recaudo_suma_acumulada_positivos_2']

audiencias = audiencias.drop(*col_eliminar)


In [ ]:
from pyspark.sql.functions import col, lit, max, when

### PASO 1: Duplicar filas con determinadas condiciones

# Definir una ventana por FILE_NUMBER
window_spec = Window.partitionBy("FILE_NUMBER")

# Crear columnas que me van a ayudar identificar las filas a duplicar
df = audiencias.withColumn("has_refund",
                   max(when((col("PRODUCT_CODE") == "CAN_INS") & (col("T_OPERATION_TYPE") == "REFUND"), 1).otherwise(0)).over(window_spec))

df = df.withColumn("has_valid_event",
                   max(when((col("Fecha_hora_evento").isNotNull()) & (col("T_OPERATION_TYPE") == "REFUND"), 1).otherwise(0)).over(window_spec))

# Combinar las condiciones en una nueva columna 'refund_count'
df = df.withColumn("refund_count", when((col("has_refund") == 1) & (col("has_valid_event") == 0), 1).otherwise(0))

# Filtrar y duplicar filas bajo condiciones específicas
df_to_duplicate = df.filter(
    (col("refund_count") == 1) &
    (col("T_OPERATION_KIND").isin(["SINGLE_ENTRY", "SIMPLE_PRODUCT", "COMPOSED_PRODUCT"])) &
    (col("Codigo_evento").isNotNull()) &
    (col('PRODUCT_CODE')!='CAN_INS')
)

# Añadir una columna nueva que identifique filas duplicadas para ambos DataFrames
df = df.withColumn("duplicated", lit(0))
duplicated_df = df_to_duplicate.withColumn("duplicated", lit(1))

# Unir las filas duplicadas al DataFrame original
final_df = df.union(duplicated_df).drop("has_refund", "has_valid_event", "refund_count")


### PASO 2: Pasar a 0 los valores de filas duplicadas
columns_to_zero = ["INSURED_AMT_ITX", "Booking_fee_sin_imp", "Booking_fee", "Recaudo", "Cortesias", "Tickets_comprados"]
for column in columns_to_zero:
    final_df = final_df.withColumn(column, when(col("duplicated") == 1, 0).otherwise(col(column)))



### PASO 3: Multiplicar *-1 INSURED_AMT_ETX
final_df = final_df.withColumn("INSURED_AMT_ETX", when(col("duplicated") == 1, col("INSURED_AMT_ETX") * -1).otherwise(col("INSURED_AMT_ETX")))



### Paso 4: Cambiar la fecha de compra (JULIAN FILTRA EL CONSOLIDADO DE MES POR ESTE CAMPO / NO SE EL RESTO DE CAMPOS SI TAMBIEN SE DEBAN DE CAMBIAR EN ALGO)
## COL 1
# Definir una ventana por FILE_NUMBER
window_spec = Window.partitionBy("FILE_NUMBER")

# Encontrar el valor de 'Fecha_hora_compra' para las filas que son 'CAN_INS' y 'Refund'
final_df = final_df.withColumn("ref_fecha_hora_compra",
                               max(when((col("PRODUCT_CODE") == "CAN_INS") & (col("T_OPERATION_TYPE") == "REFUND"),
                                          col("Fecha_Compra")).otherwise(None)).over(window_spec))

# Actualizar la columna 'Fecha_hora_compra' en las filas duplicadas
final_df = final_df.withColumn("Fecha_Compra",
                               when(col("duplicated") == 1, col("ref_fecha_hora_compra")).otherwise(col("Fecha_Compra")))

## COL 2
# Encontrar el valor de 'Fecha_hora_compra' para las filas que son 'CAN_INS' y 'Refund'
final_df = final_df.withColumn("ref_fecha_hora_compra_2",
                               max(when((col("PRODUCT_CODE") == "CAN_INS") & (col("T_OPERATION_TYPE") == "REFUND"),
                                          col("Hora_de_compra")).otherwise(None)).over(window_spec))

# Actualizar la columna 'Fecha_hora_compra' en las filas duplicadas
final_df = final_df.withColumn("Hora_de_compra",
                               when(col("duplicated") == 1, col("ref_fecha_hora_compra_2")).otherwise(col("Hora_de_compra")))


## COL 3
# Encontrar el valor de 'Fecha_hora_compra' para las filas que son 'CAN_INS' y 'Refund'
final_df = final_df.withColumn("ref_fecha_hora_compra_3",
                               max(when((col("PRODUCT_CODE") == "CAN_INS") & (col("T_OPERATION_TYPE") == "REFUND"),
                                          col("T_OPERATION_TYPE")).otherwise(None)).over(window_spec))

# Actualizar la columna 'Fecha_hora_compra' en las filas duplicadas
final_df = final_df.withColumn("T_OPERATION_TYPE",
                               when(col("duplicated") == 1, col("ref_fecha_hora_compra_3")).otherwise(col("T_OPERATION_TYPE")))


# Columnas a eliminar
col_eliminar = ['ref_fecha_hora_compra','ref_fecha_hora_compra_2','ref_fecha_hora_compra_3']

final_df = final_df.drop(*col_eliminar)

In [ ]:
# display(final_df.filter(final_df["FILE_NUMBER"]=="15534293")) #REFUND - CAMBIO EVENTO, NO SE REGRESA SEGURO                                         !!!!!!!
# display(final_df.filter(final_df["FILE_NUMBER"]=="15625275")) # COMPRO 2 EVENTOS QUE SE LLAMAN IGUAL PERO EN SESIONES DIFERENTES                ******
# display(final_df.filter(final_df['FILE_NUMBER']=='11409182')) #REFUND DE TODAS LAS BOLETAS, NO SE REGERSA EL SEGURO                 
# display(final_df.filter(final_df['FILE_NUMBER']=='16669980')) #COMPRA VARIOS EVENTOS                                                ******
# display(final_df.filter(final_df['FILE_NUMBER']=='14854919')) #REFUND DE LAS BOLETAS Y EL SEGURO                                
# display(final_df.filter(final_df['FILE_NUMBER']=='15440159'))# REFUND DE ALGUNAS BOLETAS, NO SE REGRESA EL SEGURO               *****

# display(final_df.filter(final_df["FILE_NUMBER"]=="15506697")) #VER SUMA POSITIVA  # EL TEMA DE PRODUCT COMPOSITION, NO SE SI ESO CAUSO Q SE HAYA TIRADO TODOOOOO, TOCA VER LOS EJEMPLOS A VER Q
# display(final_df.filter(final_df["FILE_NUMBER"]=="15514192")) #CASO MILLONARIOS. NO TRAE NINGUNA INFO EN EL CAMPO INSURED :,(

# display(final_df.filter(final_df['FILE_NUMBER']=='15531324')) #CASO PONDERACION DEL MISMO EVENTO PERO LOCALIDAD DISTINTA                    ******
# display(final_df.filter(final_df['FILE_NUMBER']=='15464650')) #REFUND DE ALGUNAS BOLETAS (CAMBIO DE FECHA), NO SE REGRESA PLATA DEL SEGURO              !!!!!!!




# ###NO ANOTE TODOS LOS CASOS, SON MUCHOS

In [ ]:
# ## TABLA SALES
# #inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/temp/Daniel/audiencias/Ds_G_audiencias.parquet'
# #df1 = spark.read.parquet(inpath)
# from pyspark.sql.functions import col, sum, when

# df1 = final_df.withColumn("Fecha_Compra", to_timestamp("Fecha_Compra"))

# # Filtrar para obtener solo las filas de enero
# df1 = df1.filter((month(col("Fecha_Compra")) == 3)&(year(col("Fecha_Compra"))==2024))
# # Realizar ambas agregaciones en una sola operación groupBy
# df1 = df1.groupBy("FILE_NUMBER").agg(
#     sum("INSURED_AMT_ETX").alias("suma_valor"),
#     (sum(when(col("product_code") == "CAN_INS", col("recaudo")))/1.19).alias("suma_recaudo_CAN_INS")
# )

# df1 = df1.withColumn("dif",col("suma_valor")-col("suma_recaudo_CAN_INS"))

# # Mostrar el resultado
# display(df1)

In [ ]:
# total_sum = df1.agg(sum("suma_valor").alias("suma_valor"))
# # Mostrar el resultado
# total_sum.show()


# total_sum2 = df1.agg(sum("suma_recaudo_CAN_INS").alias("suma_recaudo_CAN_INS"))
# # Mostrar el resultado
# total_sum2.show()

In [ ]:
# # Ejemplos de expedientes que no se como arreglar, casos minimos pero que por como se comportan (la fecha de compra del seguro y los eventos son distintas) sale la plata en uno u otro mes
# # La fecha esta cruzada (hago devolucion de boletas y seguro. Si esta el espacio, yo no le cambio la fecha (Este caso en particular creo q Paila)) o toca evaluar con julian y kevin

# display(audiencias.filter(audiencias['FILE_NUMBER']=='16209004')) 
# display(final_df.filter(final_df['FILE_NUMBER']=='11972945'))

## FIN AJUSTE CAMPO INSURED_AMT_ETX

In [ ]:
# tmp = audiencias.select('Sesion','TEAM_2').\
#     filter(col('Sesion') == '10229481277847').distinct()

# #display(tmp.limit(100))
# display(tmp)

In [ ]:
# 

# tmp = spark.sql(
#     """
# SELECT DISTINCT
#   Tema,
#   Tema_gral
# FROM audiencias_vw
# WHERE YEAR(Fecha_evento) IN (2020)
# """)

# #print(tmp.count()) 
# display(tmp)

In [ ]:
from pyspark.sql.functions import expr

audiencias = final_df.withColumn(
    "Fecha Actualizacion",
    expr("current_timestamp() - interval 5 hours")
)

In [ ]:
from pyspark.sql.functions import col

archivo_csv = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/temp/metodo/metodos_pago_2.csv"
df = spark.read.csv(archivo_csv, header=True, inferSchema=True, sep=';',encoding="ISO-8859-1")

# # Renombra la columna en `df` para evitar conflictos de nombres
df = df.withColumnRenamed("Metodo_de_pago", "Metodo_de_pago_2")

# Realiza un join basado en la columna "Metodo_de_pago" utilizando alias
audiencias = audiencias.alias("a").join(
    df.alias("b"),
    col("a.Metodo_de_pago") == col("b.Metodo_de_pago_2"),
    "left_outer"
)

In [ ]:
from pyspark.sql.functions import col, initcap
# Obtiene todas las columnas
todas_las_columnas = audiencias.columns

# Selecciona solo las columnas de tipo cadena
columnas_cadena = [col_name for col_name, dtype in audiencias.dtypes if dtype == "string"]

# Lista de campos que deseas omitir
campos_a_omitir = ["CORREO",'Tema',"Tema_gral",'T_OPERATION_TYPE','T_OPERATION_KIND','PRODUCT_CODE']

# # Aplica las transformaciones a las columnas de tipo cadena
# audiencias_transformado = audiencias.select(
#     *[initcap(col(column)).alias(column) if column in columnas_cadena and column != "CORREO" else col(column) for column in todas_las_columnas]
# )

# Aplica las transformaciones a las columnas de tipo cadena, omitiendo los campos especificados
audiencias_transformado = audiencias.select(
    *[initcap(col(column)).alias(column) if column in columnas_cadena and column not in campos_a_omitir else col(column) for column in todas_las_columnas]
)

In [ ]:
## MAESTRA LOCALIDADES AGRUPADAS MOVISTAR
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Maestras/Localidades_Movistar/localidades_agrupadas_movistar.csv'
localidad_movistar = spark.read.csv(inpath, sep = '|', header=True, inferSchema=True)


audiencias_transformado = audiencias_transformado.join(localidad_movistar,\
    audiencias_transformado["Categoria_Logica"] == localidad_movistar["LOGICAL_SEAT_CATEGORY"],\
    "left_outer").\
    drop('LOGICAL_SEAT_CATEGORY')

In [ ]:
# Impresion
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Audiencias/"
# 2023_fcf
join_df_coalesced = audiencias_transformado.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)


#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_G_audiencias.parquet")

True

In [ ]:
#filter((Audiencias.Codigo_organismo == '101385877267') & (year(Audiencias.Fecha_Compra) == 2019))

In [ ]:
# handling = spark.sql(
#     """
# WITH temp as (
#     SELECT
#     s.FILE_NUMBER as Expediente,
#     CONCAT(YEAR(s.REFERENCE_DATE),'_',MONTH(s.REFERENCE_DATE)) as Mes_anio,
#     --s.ORDER_NUMBER as Pedido,
#     SUM(s.NET_SOLD_TKT_AMT_ITX) as Handling_fee
#     FROM sales_vw s
#     WHERE --YEAR(s.REFERENCE_DATE) >= 2022
#         s.T_OPERATION_TYPE IN ('SALE','REFUND')
#         AND s.T_OPERATION_KIND IN ('DELIVERY_OVERHEADS','ORDER_OVERHEADS')--'COMPOSED_PRODUCT' 
#     --		AND s.PRODUCT_CODE != 'CAN_INS'
#         --AND s.product_family IN ('Paquete','Competición','Evento','Paquetes',
#             --'Pases con fecha y hora','Pase de visita','Pases sin fecha') 
#     GROUP BY
#     s.FILE_NUMBER,s.REFERENCE_DATE
# )

# SELECT
# CONCAT(Expediente,'|',Mes_anio) as Key_handling,
# SUM(Handling_fee) Handling_fee
# FROM temp
# GROUP BY Expediente,Mes_anio
# ORDER BY Expediente,Mes_anio
# """
# )

# print(handling.count())
# display(handling.limit(10))


In [ ]:
# # expedientes con mas de un numero pedido para handling:
# from pyspark.sql import Window

# # Agregar una columna que indique si es duplicado o no
# window_spec = Window().partitionBy("Expediente").orderBy("Expediente")

# df_spark = handling.withColumn("es_duplicado", count("Expediente").over(window_spec) > 1)

# # Filtrar solo las filas duplicadas
# duplicados = df_spark.filter("es_duplicado")#.select('Evento').distinct()
# #duplicados = duplicados.withColumn('Evento_N', lit('Hola'))
# # DF = df3.toPandas()

# # # Verificar duplicados en la columna 'llave_identificadora'
# # duplicados = DF[DF.duplicated(subset=['Key_evento'], keep=False)]

# #df_spark = spark.createDataFrame(duplicados)
# display(duplicados)
# print(duplicados.count())

In [ ]:
# # refund = duplicados.select('Expediente','Handling_fee').\
# #     groupBy("Expediente").sum("Handling_fee").\
# #         filter(col('sum(Handling_fee)') != 0)

# # display(refund)
# # print(refund.count())

# handling = sales.filter(col('T_OPERATION_KIND').isin('DELIVERY_OVERHEADS','ORDER_OVERHEADS')).\
#     filter(col('T_OPERATION_TYPE').isin('SALE','REFUND')).\
#     select('FILE_NUMBER','NET_SOLD_TKT_AMT_ITX').\
#     groupBy("FILE_NUMBER").\
#         agg(sum("NET_SOLD_TKT_AMT_ITX").alias("Handling_fee")).\
#     orderBy(desc("Handling_fee"))

# #display(handling)
# print(handling.count())

In [ ]:
# # Impresion
# # Guarda el DataFrame en formato Parquet en Azure Blob Storage
# inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Handling_fee/"
# # 2023_fcf
# join_df_coalesced = handling.coalesce(1)
# join_df_coalesced.write.mode("overwrite").parquet(inpath)

# #cambio de nombre
# files=dbutils.fs.ls(inpath)
# output_file= [x for x in files if x.name.startswith("part-")]

# # EN la misma carpeta de origen
# dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_G_handling.parquet")